In [1]:
from url_validator import URLValidator
from api_keys import serpapi_key

/home/joshua/.local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2025-02-14 21:18:32.730642: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-14 21:18:32.834115: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-14 21:18:34.650833: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Unit Test
Identical Prompt: "NVIDIA stock"

Note that some scores (namely Content Relevance) are subject to change as some of these pages should update daily.

Fixed Scores:
- Domain Trust is always 60 because it currently has a placeholder value.
- Fact-Check Score is always 50 because the Fact-Check scorer uses a malformed API endpoint and thus always fails to obtain data that can be converted to json format, defaulting to 50.
- Bias Score appears to always return "Negative" for sentiment, resulting in a score of 30.
- Citation Score should almost always be 0, as nearly all webpages are not in academic papers.

In [2]:
validator = URLValidator(serpapi_key)
user_prompt = "NVIDIA stock"

## Forbes Magazine Article
Content Relevance: 43.12271177768707

Note that the longer the text of a page, the more likely it contains content that is unrelated to the prompt, which causes the content relevance score to reduce. This is not particularly desirable, as long-form content such as articles are more likely to be penalized than shorter webpages. The semantic comparison engine may need to be adjusted based on length of the webpage content.

In [4]:
url_to_check = "https://www.forbes.com/sites/dereksaul/2025/01/28/deepseek-panic-live-updates-nvidia-stock-makes-history-again-with-260-billion-rebound/"

validator.rate_url_validity(user_prompt, url_to_check)

{'raw_scores': {'content_relevance': 4.97,
  'domain_trust': 4.28,
  'title_relevance': 5.0,
  'final_score': 4.75},
 'stars': {},
 'explanation': ''}

## CNBC Stock Quotes
Content Relevance: 8.290518075227737

Notice that the "PROFILE" section on this webpage has not been picked up by this scraper, removing most of the relevant text from the page that would be related to the user prompt. This is a deficiency in the scraper which needs to be rectified. This website is highly relevant to the user prompt and is scored extremely low on that metric.

In [5]:
%%time
url_to_check = "https://www.cnbc.com/quotes/NVDA"

validator.rate_url_validity(user_prompt, url_to_check)

CPU times: user 1min 8s, sys: 95.9 ms, total: 1min 8s
Wall time: 17.5 s


{'raw_scores': {'content_relevance': 5.0,
  'domain_trust': 4.5,
  'title_relevance': 1.0,
  'final_score': 3.5},
 'stars': {},
 'explanation': ''}

## Official NVIDIA Website
Content Relevance: 41.894444823265076

Note that the content relevance is matching heavily on the word "NVIDIA", even though this homepage has nothing to do with stocks. A content relevance score of ~42 is not excessively high, but it is nearly the same as the Forbes' article which is specifically about NVIDIA stocks, despite this page having nothing to do with stocks. This may be a drawback of the semantic comparison engine.

In [6]:
url_to_check = "https://www.nvidia.com"

validator.rate_url_validity(user_prompt, url_to_check)

{'raw_scores': {'content_relevance': 3.95,
  'domain_trust': 1.0,
  'title_relevance': 2.5,
  'final_score': 2.48},
 'stars': {},
 'explanation': ''}

## Marketwatch
Error from being unable to scrape a highly reputable website designed with anti-scraping measures. It may be worthwhile to rate the domain credibility separately from trying to scrape the website.

In [7]:
url_to_check = "https://www.marketwatch.com/investing/stock/nvda"

validator.rate_url_validity(user_prompt, url_to_check)

Failed to fetch content from https://www.marketwatch.com/investing/stock/nvda: 401 Client Error: HTTP Forbidden for url: https://www.marketwatch.com/investing/stock/nvda
Content relevance score and outgoing link credibility cannot be calculated.


{'raw_scores': {'domain_trust': 5.0,
  'title_relevance': 1.0,
  'final_score': 3.0},
 'stars': {},
 'explanation': ''}

## Yahoo Finance
Content Relevance: 71.57930135726929

The web scraper managed to pick up all of the information in the "About" section of the webpage and little else, giving a high performance in terms of content relevance.

In [8]:
url_to_check = "https://finance.yahoo.com/quote/NVDA/"

validator.rate_url_validity(user_prompt, url_to_check)

{'raw_scores': {'content_relevance': 4.28,
  'domain_trust': 5.0,
  'title_relevance': 1.0,
  'final_score': 3.43},
 'stars': {},
 'explanation': ''}

## Trading View
Content Relevance: 34.80248749256134

The content of this page is highly relevant towards the actual stock price and history of NVIDIA but contains very little text for the scraper to capture. This causes the content relevance to drop when it should be fairly high.

In [9]:
url_to_check = "https://www.tradingview.com/symbols/NASDAQ-NVDA/"

validator.rate_url_validity(user_prompt, url_to_check)

{'raw_scores': {'content_relevance': 5.0,
  'domain_trust': 4.0,
  'title_relevance': 1.0,
  'final_score': 3.33},
 'stars': {},
 'explanation': ''}

## Barchart Article
Content Relevance: 20.299533009529114

This article provides useful information about NVIDIA stock, but the scraper failed to pick up nearly any of the text available on the page.

In [10]:
url_to_check = "https://www.barchart.com/story/news/30841907/dear-nvidia-stock-fans-mark-your-calendars-for-february-26"

validator.rate_url_validity(user_prompt, url_to_check)

{'raw_scores': {'content_relevance': 1.0,
  'domain_trust': 4.0,
  'title_relevance': 5.0,
  'final_score': 3.33},
 'stars': {},
 'explanation': ''}

## CNN Stock Lookup
Content Relevance: 62.75913119316101

The scraper picked up on the "About NVDA" section which boosted the content relevance score.

In [11]:
url_to_check = "https://www.cnn.com/markets/stocks/NVDA"

validator.rate_url_validity(user_prompt, url_to_check)

{'raw_scores': {'content_relevance': 4.23,
  'domain_trust': 4.0,
  'title_relevance': 1.0,
  'final_score': 3.08},
 'stars': {},
 'explanation': ''}

## Google Finance Stock Quote
Content Relevance: 9.07139778137207

Like the CNBC stock quote website, the scraper failed to pick up the "About" text on the page. In fact, the scraper picked up no text at all from the page, so it is surprising that the content relevance is nonzero.

In [12]:
url_to_check = "https://www.google.com/finance/quote/NVDA:NASDAQ?hl=en"

validator.rate_url_validity(user_prompt, url_to_check)

{'raw_scores': {'content_relevance': 4.26,
  'domain_trust': 3.29,
  'title_relevance': 1.0,
  'final_score': 2.85},
 'stars': {},
 'explanation': ''}

# Negative Unit Tests (deliberately wrong URLs)

## Yahoo Finance Quote for AMD
Content Relevance: 34.59650278091431

Although the relevance score is not particularly high, it performs about average for most of the other unit tests involving correct URLs referencing NVIDIA, even though this URL is focused on AMD. This is probably because the semantic scorer is matching on the "stock" part of the prompt.

In [13]:
url_to_check = "https://finance.yahoo.com/quote/AMD/"

validator.rate_url_validity(user_prompt, url_to_check)

{'raw_scores': {'content_relevance': 1.7,
  'domain_trust': 5.0,
  'title_relevance': 1.0,
  'final_score': 2.57},
 'stars': {},
 'explanation': ''}

## WebMD Page on COVID-19
Content Relevance: 13.424356281757355

This page has nothing to do with NVIDIA or stocks, and yet it has scored higher than many relevant links!

In [14]:
url_to_check = "https://www.webmd.com/covid/coronavirus"

validator.rate_url_validity(user_prompt, url_to_check)

{'raw_scores': {'content_relevance': 1.0,
  'domain_trust': 1.0,
  'title_relevance': 1.0,
  'final_score': 1.0},
 'stars': {},
 'explanation': ''}